In [17]:
import pandas as pd
import numpy as np
import shutil
import os
import datetime

riverCol = 'Major River'
elementCol = 'Network Element'

#Constituents = ['Sediment - Fine', 'Sediment - Coarse', 'N_Particulate', 'N_DIN', 'N_DON', 'P_Particulate', 'P_DOP', 'P_FRP']
Constituents = ['Ametryn', 'S-metolachlor', 'Atrazine', 'Diuron', 'Hexazinone', 'Tebuthiuron', '24-D'
                , 'Paraquat', 'Glyphosate', 'Chlorsulfuron', 'Diquat', 'Fluroxypyr', 'Haloxyfop', 'Imazapic'
               , 'Imazethapyr', 'Isoxaflutole', 'MCPA', 'Metribuzin', 'Metsulfuron-methyl', 'Pendimethalin', 'Simazine'
               , 'Terbuthylazin', 'Trifluralin', 'Aciflurofen', 'Chlorpyrifos', 'Fipronil', 'Imidacloprid', 'Prometryn', 'Triclopyr']

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BurdekinElementMapper.csv'
#mainPath = r'P:\projects\MichaelWarne\Models\results\Burdekin Rebuild 2014\BU_msPAF_Calibrated_503\TimeSeries'
#outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\Burdekin'

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BurnettMaryElementMapper.csv'
#mainPath = r'C:\DDrive\RC10PesticideProjs\results\Burnett Mary Rebuild 2014\BM_RC10_MSPAF\TimeSeries'
#outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\BurnettMary'

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\FitzroyElementMapper.csv'
#mainPath = r'C:\DDrive\RC10PesticideProjs\results\FI RC2018\FI_RC10_msPAF\TimeSeries'
#outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\Fitzroy'

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\MackayWhitsundayElementMapper.csv'
#mainPath = r'C:\DDrive\RC10PesticideProjs\results\MW_DS_2015\MW_Rc10_msPAF\TimeSeries'
#outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\MackayWhitsunday'

elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\WetTropicsElementMapper.csv'
mainPath = r'C:\DDrive\RC10PesticideProjs\results\WetTropics_DS_2015\WT_RC10_msPAF\TimeSeries'
outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\WetTropics'

In [18]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

Made dir: C:\DDrive\RC10PesticideProjs\CSIROAggregated\WetTropics


In [19]:
RiverElements = {}
elementsTable = pd.read_csv(elementMapperCSV)

for index, row in elementsTable.iterrows():
    if not row[riverCol] in RiverElements.keys():
        RiverElements[row[riverCol]] = []
    
    RiverElements[row[riverCol]].append(row[elementCol])


#print(RiverElements)

In [20]:
#Set up the totals first
riverTotals = {}
for theRiver in RiverElements.keys():
    riverTotals[theRiver] = pd.DataFrame()

#Now data

theFlowsDIR = os.path.join(mainPath,'Flows')
for theRiver in RiverElements.keys():
    #We'll want flows first
    eleCount = 0
    flowFrame = pd.DataFrame()
    outputFields = ['Date']
    for theElement in RiverElements[theRiver]:
        for fname in os.listdir(theFlowsDIR):
            if theElement in fname:
                #print("For this river: " + theRiver + " we have this file: " + fname)
                flowFilePath = os.path.join(theFlowsDIR,fname)
                flowData = pd.read_csv(flowFilePath)
                flowData.columns = ['Date', theElement]
                #riverTotals[theRiver].columns.drop(theElement)
                outputFields.append(theElement)
                if eleCount == 0:
                    #Get the structure for our totals first
                    riverTotals[theRiver] = flowData
                    riverTotals[theRiver].drop(theElement, axis=1, inplace=True)
                    #Now process stuff
                    flowFrame = flowData
                    #flowFrame.columns = ['Date', theElement]
                    #flowFrame['Date'] = flowData['Date']
                    #flowFrame[theElement] = flowData.iloc[:1].values#This sould give the 2nd column
                else:
                    #Join...??
                    #flowFrame[theElement] = flowData.iloc[:1].values
                    flowFrame = pd.merge(flowFrame, flowData, how='left', left_on=['Date'], right_on=['Date'])
                
                eleCount += 1
    #Totals
    #The .iloc[:, 1:] should exclude our first col, Date, from the sum
    #flowFrame['Flow_cumecs'] = flowFrame.iloc[:, 1:].sum(axis=1)
    flowFrame['Flow_cumecs'] = flowFrame.sum(axis=1)
    #Write out flow
    theOutFlowFile = os.path.join(outPath, "Flow " + theRiver + " cubicmetrespersecond.csv")
    flowFrame.to_csv(theOutFlowFile, index=False)
    #Need to join flow Total to RiverTotals
    riverTotals[theRiver] = pd.merge(riverTotals[theRiver], flowFrame[['Date', 'Flow_cumecs']], how='left', left_on=['Date'], right_on=['Date'])

#Now water quality
for theWQBit in Constituents:
    wqPath = os.path.join(mainPath, theWQBit)
    if not os.path.exists(wqPath):
        continue
    
    for theRiver in RiverElements.keys():
        #We'll want flows first
        eleCount = 0
        wqFrame = pd.DataFrame()
        outputFields = ['Date']
        for theElement in RiverElements[theRiver]:
            for fname in os.listdir(wqPath):
                if theElement in fname:
                    #print("For this river: " + theRiver + " we have this file: " + fname)
                    wqFilePath = os.path.join(wqPath,fname)
                    wqData = pd.read_csv(wqFilePath)
                    wqData.columns = ['Date', theElement]
                    outputFields.append(theElement)
                    if eleCount == 0:
                        wqFrame = wqData
                    else:
                        wqFrame = pd.merge(wqFrame, wqData, how='left', left_on=['Date'], right_on=['Date'])
                
                    eleCount += 1
        #Totals
        #wqFrame[theRiver + '_kg'] = wqFrame.iloc[:, 1:].sum(axis=1)
        wqFrame[theRiver + '_kg'] = wqFrame.sum(axis=1)
        #Write out flow
        theOutFlowFile = os.path.join(outPath, theWQBit + " " + theRiver + " kilograms.csv")
        wqFrame.to_csv(theOutFlowFile, index=False)
        #Join to totals
        #rename constituent total first
        wqFrame.rename(columns={theRiver + '_kg':theWQBit + '_kg'}, inplace=True)
        riverTotals[theRiver] = pd.merge(riverTotals[theRiver], wqFrame[['Date', theWQBit + '_kg']], how='left', left_on=['Date'], right_on=['Date'])


#Now write totals files
for theRiver in RiverElements.keys():
    theOutTotalsFile = os.path.join(outPath, theRiver + " Totals.csv")
    riverTotals[theRiver].to_csv(theOutTotalsFile, index=False)


print("All Done")

All Done
